In [107]:
import pickle
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.datasets import make_moons
from sklearn.decomposition import PCA
from scipy.linalg import fractional_matrix_power

np.random.default_rng(123).shuffle(random_index)

def load_data(file):
    data_list = []
    label_list = []
    for i in range(1,6):
        with open(f"{file}/data_batch_{i}", 'rb') as f:
            _data = pickle.load(f, encoding="bytes")
            #import pdb; pdb.set_trace()
            data_list.append(_data[b'data'])
            label_list.append(_data[b'labels'])
    return (data_list, label_list)
            
data_cifar10 = load_data("./cifar10")

In [108]:
data, label = data_cifar10
data = np.concatenate(data)
label = np.concatenate(label)
data= (data - np.mean(data, axis=1)[:,np.newaxis])/(np.var(data, axis=1)[:,np.newaxis])

selected_labels = [2, 4, 8, 9]

selected_data = np.zeros(data.shape[0]).astype(bool)

for l in selected_labels:
    selected_data = np.bitwise_or(selected_data, label == l)
    

data = data[selected_data]
label[label==2] -= 2
label[label==4] -= 3
label[label==8] -= 6
label[label==9] -= 6
label = label[selected_data]

In [109]:

alpha = 0.99
sigma = 0.1
num_iter = 1000
num_label_per_cls = 10

num_data = data.shape[0]

In [110]:
label[100:200]

array([3, 0, 1, 3, 3, 3, 2, 0, 0, 0, 2, 2, 3, 1, 1, 0, 0, 3, 3, 2, 1, 3,
       3, 3, 2, 0, 1, 3, 1, 1, 1, 2, 0, 3, 0, 3, 1, 2, 3, 1, 1, 2, 1, 0,
       3, 1, 2, 1, 1, 0, 0, 0, 3, 3, 2, 0, 1, 2, 3, 1, 0, 0, 0, 3, 1, 2,
       1, 1, 3, 2, 2, 3, 3, 1, 1, 2, 1, 2, 2, 0, 2, 3, 0, 2, 1, 3, 0, 1,
       2, 2, 1, 1, 3, 2, 2, 0, 2, 1, 2, 3])

In [116]:
random_index = np.arange(num_data)
np.random.shuffle(random_index)
data = data[random_index]
label = label[random_index]

In [112]:
def pca_dim_reduce(X, output_dim=20):
    
    pca = PCA(n_components = output_dim)
    newData = pca.fit_transform(X)
    return newData



def generate_affinity_matrix(X, Sigma=0.01,metric="L2"):
    W = np.sum((X[np.newaxis,:,:] - X[:,np.newaxis,:])**2, axis=-1)
    #import pdb;pdb.set_trace()
    W = np.exp(- W / (2*Sigma**2))
    np.fill_diagonal(W, 0)
    D_n12 = np.sum(W, axis=1)**(-1/2)
    D_n1 = np.sum(W, axis=1)**(-1)
    D_n12 = np.diag(D_n12)
    D_n1 = np.diag(D_n1)
    return W, D_n12, D_n1

def walk_matrix(X, Sigma=0.01, m_type='norm'):
    if m_type == 'norm':
        W, D_n12, _ = generate_affinity_matrix(X, Sigma=Sigma)
        S = D_n12 @ W @ D_n12
        return S
    elif m_type == 'random':
        W, _, D_n1 = generate_affinity_matrix(X, Sigma=Sigma)
        S = W @ D_n1
        return  S
    else:
        raise NotImplemented()

def label_matrix(label_vec):
    #one-hot
    n_values = np.max(label_vec) + 1
    one_hot_y = np.eye(n_values)[label_vec]
    return one_hot_y

        
def lgc(S, Y, alpha=0.99,num_I=400):
    
    # Iteration 0/ F_0 = Y
    F = Y
    for t in range(num_I):
        F = S @ F * alpha + (1-alpha)*Y
    
    return F



In [113]:
from time import time
import pandas as pd
#data_exp = data

In [92]:
data_dict = {"Label Number":[], "Dim Number":[], "Time":[], "Acc":[]}
for dim in (8, 16, 32, 64, 128):
    data_exp = pca_dim_reduce(data, output_dim=dim)
    for labeled_num in (10, 20, 50, 100, 200, 500):
        sample_num = 5000


        _s = time()
        Y = np.zeros([sample_num, 4])
        data_exp = data_exp[:sample_num]
        ##data_exp = (data[:sample_num] - np.mean(data[:sample_num]))/(np.var(data[:sample_num]))
        target = label[:sample_num]
        matrix = label_matrix(label[:labeled_num])
        Y[:labeled_num] = matrix
        S = walk_matrix(data_exp, Sigma=sigma, m_type='norm')
        F = lgc(S, Y, alpha, num_iter)

        pred_y = np.argmax(F, axis=-1)
        data_dict["Label Number"].append(labeled_num)
        data_dict["Dim Number"].append(dim)
        print(f"For {labeled_num} Labels")
        print(f"with {dim} DIMS")
        acc = np.mean(pred_y[labeled_num:] == target[labeled_num:])
        data_dict["Acc"].append(acc)
        data_dict['Time'].append(time() - _s)
        print(f"Acc{data_dict['Acc'][-1]}")
        print(f"Time{data_dict['Time'][-1]}")

out_df = pd.DataFrame.from_dict(data_dict)
out_df.to_csv('label-dim-2489.csv', index=False)


KeyboardInterrupt: 

In [100]:
labeled_num = 500
data_dict = {"Alpha":[], "Sigma":[], "Time":[], "Acc":[]}
data_exp = pca_dim_reduce(data, output_dim=64)
sample_num = 5000
Y = np.zeros([sample_num, 4])
data_exp = data_exp[:sample_num]
##data_exp = (data[:sample_num] - np.mean(data[:sample_num]))/(np.var(data[:sample_num]))
target = label[:sample_num]
matrix = label_matrix(label[:labeled_num])
Y[:labeled_num] = matrix

for a_f in [0.999, 0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.01, 0.001]:
    for s_a in [100, 10, 1, 0.1, 0.01, 0.001, 0.0001]:
        _s = time()
        S = walk_matrix(data_exp, Sigma=s_a, m_type='norm')
        F = lgc(S, Y, a_f, num_I=2000)
        pred_y = np.argmax(F, axis=-1)
        data_dict["Alpha"].append(a_f)
        data_dict["Sigma"].append(s_a)
        print(f"For {a_f} MIU")
        print(f"with {s_a} Sigma")
        #acc = np.mean(pred_y == target)
        acc = np.mean(pred_y[labeled_num:] == target[labeled_num:])
        data_dict["Acc"].append(acc)
        data_dict['Time'].append(time() - _s)
        print(f"Acc{data_dict['Acc'][-1]}")
        print(f"Time{data_dict['Time'][-1]}")

out_df = pd.DataFrame.from_dict(data_dict)
out_df.to_csv('miu-sigma-64-500-remove.csv', index=False)

For 0.999 MIU
with 100 Sigma
Acc0.25244444444444447
Time18.273845911026
For 0.999 MIU
with 10 Sigma
Acc0.25244444444444447
Time18.304597854614258
For 0.999 MIU
with 1 Sigma
Acc0.24155555555555555
Time18.292303800582886
For 0.999 MIU
with 0.1 Sigma
Acc0.49
Time18.310458421707153


/home/mooler/anaconda3/envs/pp/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in power
  
/home/mooler/anaconda3/envs/pp/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in reciprocal
  from ipykernel import kernelapp as app
/home/mooler/anaconda3/envs/pp/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in reciprocal
  from ipykernel import kernelapp as app
/home/mooler/anaconda3/envs/pp/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in matmul


For 0.999 MIU
with 0.01 Sigma
Acc0.25244444444444447
Time18.2416090965271
For 0.999 MIU
with 0.001 Sigma
Acc0.25244444444444447
Time18.273045539855957
For 0.999 MIU
with 0.0001 Sigma
Acc0.25244444444444447
Time18.243876218795776
For 0.99 MIU
with 100 Sigma
Acc0.25244444444444447
Time18.386842250823975
For 0.99 MIU
with 10 Sigma
Acc0.25244444444444447
Time18.286447048187256
For 0.99 MIU
with 1 Sigma
Acc0.24155555555555555
Time18.311567544937134
For 0.99 MIU
with 0.1 Sigma
Acc0.5257777777777778
Time18.2831609249115
For 0.99 MIU
with 0.01 Sigma
Acc0.25244444444444447
Time18.255253791809082
For 0.99 MIU
with 0.001 Sigma
Acc0.25244444444444447
Time18.244733572006226
For 0.99 MIU
with 0.0001 Sigma
Acc0.25244444444444447
Time18.250364065170288
For 0.9 MIU
with 100 Sigma
Acc0.25244444444444447
Time18.26825451850891
For 0.9 MIU
with 10 Sigma
Acc0.25244444444444447
Time18.302055835723877
For 0.9 MIU
with 1 Sigma
Acc0.24155555555555555
Time18.280114889144897
For 0.9 MIU
with 0.1 Sigma
Acc0.556
Ti

In [106]:
labeled_num = 500
data_dict = {"Alpha":[], "Sigma":[], "Time":[], "Acc":[]}
data_exp = pca_dim_reduce(data, output_dim=64)
sample_num = 5000
Y = np.zeros([sample_num, 4])
data_exp = data_exp[:sample_num]
##data_exp = (data[:sample_num] - np.mean(data[:sample_num]))/(np.var(data[:sample_num]))
target = label[:sample_num]
matrix = label_matrix(label[:labeled_num])
Y[:labeled_num] = matrix

for a_f in [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]:
    for s_a in [0.6,0.5,0.4, 0.3, 0.2, 0.08, 0.05]:
        _s = time()
        S = walk_matrix(data_exp, Sigma=s_a, m_type='random')
        F = lgc(S, Y, a_f, num_I=2000)
        pred_y = np.argmax(F, axis=-1)
        data_dict["Alpha"].append(a_f)
        data_dict["Sigma"].append(s_a)
        print(f"For {a_f} MIU")
        print(f"with {s_a} Sigma")
        #acc = np.mean(pred_y == target)
        acc = np.mean(pred_y[labeled_num:] == target[labeled_num:])
        data_dict["Acc"].append(acc)
        data_dict['Time'].append(time() - _s)
        print(f"Acc{data_dict['Acc'][-1]}")
        print(f"Time{data_dict['Time'][-1]}")

out_df = pd.DataFrame.from_dict(data_dict)
out_df.to_csv('miu-sigma-64-500-random-fine.csv', index=False)

For 0.9 MIU
with 0.6 Sigma
Acc0.2851111111111111
Time17.958566427230835
For 0.9 MIU
with 0.5 Sigma
Acc0.34444444444444444
Time17.99203848838806
For 0.9 MIU
with 0.4 Sigma
Acc0.5031111111111111
Time18.00738525390625
For 0.9 MIU
with 0.3 Sigma
Acc0.5646666666666667
Time17.994711875915527
For 0.9 MIU
with 0.2 Sigma
Acc0.5877777777777777
Time18.00091052055359
For 0.9 MIU
with 0.08 Sigma
Acc0.5153333333333333
Time17.985530376434326


/home/mooler/anaconda3/envs/pp/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in power
  
/home/mooler/anaconda3/envs/pp/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in reciprocal
  from ipykernel import kernelapp as app
/home/mooler/anaconda3/envs/pp/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in matmul


For 0.9 MIU
with 0.05 Sigma
Acc0.25244444444444447
Time18.074543237686157
For 0.8 MIU
with 0.6 Sigma
Acc0.42288888888888887
Time17.97937250137329
For 0.8 MIU
with 0.5 Sigma
Acc0.5377777777777778
Time18.032748222351074
For 0.8 MIU
with 0.4 Sigma
Acc0.5666666666666667
Time17.990366220474243
For 0.8 MIU
with 0.3 Sigma
Acc0.5902222222222222
Time18.031774759292603
For 0.8 MIU
with 0.2 Sigma
Acc0.5962222222222222
Time17.993799924850464
For 0.8 MIU
with 0.08 Sigma
Acc0.5106666666666667
Time18.13818883895874
For 0.8 MIU
with 0.05 Sigma
Acc0.25244444444444447
Time18.01954436302185
For 0.7 MIU
with 0.6 Sigma
Acc0.5313333333333333
Time18.040111780166626
For 0.7 MIU
with 0.5 Sigma
Acc0.5644444444444444
Time18.045737743377686
For 0.7 MIU
with 0.4 Sigma
Acc0.5835555555555556
Time18.059346199035645
For 0.7 MIU
with 0.3 Sigma
Acc0.5948888888888889
Time17.977024793624878
For 0.7 MIU
with 0.2 Sigma
Acc0.5968888888888889
Time18.026623964309692
For 0.7 MIU
with 0.08 Sigma
Acc0.5106666666666667
Time17.9988

In [120]:

data_dict = {"Number":[], "Time":[], "Acc":[], 'Matrix':[]}
data_exp = pca_dim_reduce(data, output_dim=64)
sample_num = 5000
Y = np.zeros([sample_num, 4])
data_exp = data_exp[:sample_num]
##data_exp = (data[:sample_num] - np.mean(data[:sample_num]))/(np.var(data[:sample_num]))
target = label[:sample_num]


for mx_type in ['norm', 'random']:
    for labeled_num in [4, 20, 40, 100, 200, 300, 400, 500]:
        matrix = label_matrix(label[:labeled_num])
        Y[:labeled_num] = matrix
        _s = time()
        S = walk_matrix(data_exp, Sigma=0.2, m_type=mx_type)
        F = lgc(S, Y, 0.3, num_I=5)
        pred_y = np.argmax(F, axis=-1)
        data_dict["Number"].append(labeled_num)
        data_dict["Matrix"].append(mx_type)
        print(f"For {labeled_num} labels")
        print(f"with {mx_type} Matrix ")
        #acc = np.mean(pred_y == target)
        acc = np.mean(pred_y[labeled_num:] == target[labeled_num:])
        data_dict["Acc"].append(acc)
        data_dict['Time'].append(time() - _s)
        print(f"Acc{data_dict['Acc'][-1]}")
        print(f"Time{data_dict['Time'][-1]}")

out_df = pd.DataFrame.from_dict(data_dict)
out_df.to_csv('Label.csv', index=False)

For 4 labels
with norm Matrix 
Acc0.3324659727782226
Time5.6690592765808105
For 20 labels
with norm Matrix 
Acc0.3791164658634538
Time5.709295749664307
For 40 labels
with norm Matrix 
Acc0.49213709677419354
Time5.627018213272095


KeyboardInterrupt: 

In [ ]:
labeled_num = 500
data_dict = {"Iter":[], "Matrix":[], "Time":[], "Acc":[]}
data_exp = pca_dim_reduce(data, output_dim=64)
sample_num = 5000
Y = np.zeros([sample_num, 4])
data_exp = data_exp[:sample_num]
##data_exp = (data[:sample_num] - np.mean(data[:sample_num]))/(np.var(data[:sample_num]))
target = label[:sample_num]
matrix = label_matrix(label[:labeled_num])
Y[:labeled_num] = matrix

for iternum in [1,2,3,4, 5, 10, 20]:
    _s = time()
    S = walk_matrix(data_exp, Sigma=0.2, m_type='norm')
    F = lgc(S, Y, 0.3, num_I=iternum)
    pred_y = np.argmax(F, axis=-1)
    data_dict["Iter"].append(iternum)
    data_dict["Matrix"].append('norm')
    print(f"Iter{data_dict['Iter'][-1]}")
    acc = np.mean(pred_y[labeled_num:] == target[labeled_num:])
    data_dict["Acc"].append(acc)
    data_dict['Time'].append(time() - _s)
    print(f"Acc{data_dict['Acc'][-1]}")
    print(f"Time{data_dict['Time'][-1]}")

    _s = time()
    S = walk_matrix(data_exp, Sigma=0.2, m_type='random')
    F = lgc(S, Y, 0.3, num_I=iternum)
    pred_y = np.argmax(F, axis=-1)
    data_dict["Iter"].append(iternum)
    data_dict["Matrix"].append('random')
    print(f"Iter{data_dict['Iter'][-1]}")
    acc = np.mean(pred_y[labeled_num:] == target[labeled_num:])
    data_dict["Acc"].append(acc)
    data_dict['Time'].append(time() - _s)
    print(f"Acc{data_dict['Acc'][-1]}")
    print(f"Time{data_dict['Time'][-1]}")

out_df = pd.DataFrame.from_dict(data_dict)
out_df.to_csv('iter-acc.csv', index=False)